In [1]:
#this scripts takes the output from var_builder
#converts timedelta to numeric
#calculates backward citation average delay
#calculates forward citation average delay
#counts forward and backward citations

# Jan 19, 2020
# Joining is not generating matches
# probably is an issue with dtype
# i am running joining with inner but it should be updated later with outer.

import pandas as pd
import numpy as np
import re
import dask.dataframe as dd
import gzip

src='data/var_builder.csv.gz'
cit_received_delay = 'data/cit_received_delay.parquet.gz'
cit_made_delay='data/cit_made_delay.csv.gz'

cols=['patent_id','citation_id', 'cit_delay' ]
dtypes={'patent_id': object, 'citation_id': object, 'cit_delay': np.float64}

In [2]:
df = dd.read_parquet('data/var_builder*.parquet.gz')

In [3]:
df.groupby(df.index).cit_delay.mean().reset_index().compute().to_parquet(cit_received_delay, compression='gzip')
# output: patent-level data
# if group by citation_id, the delay will give the average delay gives how much time it took,
#    in average, for a patent to receive citations
# it is a measure of time to become influent

# received_delay=df.groupby('citation_id').mean()
# received_delay.head()
# received_delay.dropna(inplace=True)
# received_delay.dtypes #this will tell us if i am trying to join data of the same format
# received_delay.hist()
# received_delay[received_delay["cit_delay"]<received_delay["cit_delay"].quantile(0.01)]
# received_delay[received_delay["cit_delay"]>received_delay["cit_delay"].quantile(0.99)]
# received_delay.to_csv(cit_received_delay, compression='gzip')

# if you group by patent_id, the delay will give a measure of how far back the patent is rooted.
# made_delay=df.groupby('patent_id').mean()
# made_delay.head()
# made_delay.dropna(inplace=True)
# made_delay[made_delay["cit_delay"]<made_delay["cit_delay"].quantile(0.01)].sort_values(by=['cit_delay'], ascending=False)
# made_delay[made_delay["cit_delay"]>made_delay["cit_delay"].quantile(0.99)].sort_values(by=['cit_delay'], ascending=True)

# made_delay.to_csv(cit_made_delay, compression='gzip')

In [4]:
df=df.groupby(df.patent_id).cit_delay.mean().reset_index().compute().to_parquet(cit_made_delay, compression='gzip')